In [ ]:
%use plotly
import kotlinx.coroutines.*

val (zoneData, udata) = runBlocking {
    val zoneData = async {
        analyzeZoneFiles("./data/zones")
    }
    val uData1 = async {
        readAllUData("./data/log.anon")
    }
    val uData2 = async {
        readAllUData("./data/log2.anon")
    }
    val uData = uData1.await() + uData2.await()
    Pair(zoneData.await(), uData)
}

In [ ]:
import java.net.IDN

val sortedZones = zoneData.sortedBy { r -> r.second.necessarySignatures }
val zoneStrings = sortedZones.map { (zone, _) -> IDN.toUnicode(zone) }

Plotly.plot {
    bar {
        x.strings = zoneStrings
        y.numbers = sortedZones.map { (_, stat) -> stat.necessarySignatures }
        type = TraceType.bar
    }

    layout {
        width = 1500
        height = 950
        title {
            text = "Signatures per zone"
        }
        xaxis {
            title {
                text = "Zone"
            }
        }
        yaxis {
            title {
                text = "Signatures"
                type = AxisType.log
            }
        }

    }
}

In [ ]:
import java.net.IDN

val rootZoneString = "root"

val domainsWithValidations =
    udata.map { ud -> ud.domainName }.distinct().map { domainName ->
        if (domainName == listOf("")) {
            rootZoneString
        } else {
            domainName.dropLast(1).joinToString(".")
        }
    }

fun plot(zones: List<Pair<String, ZoneStat>>, yAxisValues: List<String>): Plot {
    val zoneStrings = zones.map { (zone, _) -> IDN.toUnicode(zone) }

    val zoneMinutes = zones.map { (_, stat) -> stat.necessarySignatures * (0.424 * (1 - 0.085)) / 60 }
    println("Signed within 30 minutes: " + zoneMinutes.takeWhile { i -> i <= 30 }.size + "/" + zones.size)
    return Plotly.plot {
        bar {
            showlegend = false
            x.strings = zoneStrings
            y.numbers = zoneMinutes
            marker {
                colors(zoneStrings.map { zoneString ->
                    if (domainsWithValidations.contains(zoneString)) {
                        "rgba(21,119,180,1)"
                    } else {
                        "rgba(204,204,204,1)"

                    }
                })
            }
        }

        scatter {
            showlegend = false
            x.strings = listOf(zoneStrings.first(), zoneStrings.last())
            y.numbers = listOf(30, 30)
            mode = ScatterMode.lines
            name = "30 minutes"
        }

        layout {
            width = 1500 * 0.5
            height = 950 * 0.5
            title {
                text = "Time to sign zone"
            }
            xaxis {
                title {
                    text = "Zone"
                }
                tickvals = ListValue(
                    yAxisValues.filter { d -> domainsWithValidations.contains(d) }.map { d -> d.asValue() }
                ).list
            }
            yaxis {
                title {
                    text = "Time"
                }
                type = AxisType.log
                tickmode = TickMode.array
                tickvals = ListValue(0.1, 1, 10, 30, 100, 1020, 10080, 99360).list
                ticktext =
                    ListValue(
                        "6 seconds",
                        "1 minute",
                        "10 minutes",
                        "30 minutes",
                        "1,5 hour",
                        "17 hours",
                        "7 days",
                        "69 days"
                    ).list
            }
        }
    }
}
plot(
    sortedZones, listOf(
        "com", "amsterdam", "gov",
        "education", "support", "city", "brussels", "mov",
        "science", "new", "dance", "domains",
        "press", "adudhabi", "ryukyu", "onl", "africa", "vegas",
        "irish", "zuerich", "adult", "boston", "sucks", "youtube",
        "music", "apple", "honda", "aaa", "aws", "imdb", "now", "deal", "wow",
        "bbc", "dell", "hbo", "safety", "yandex", "playstation", "box", "cern",
        "data", "meme", "stockholm", "tui", "map", "abc", "microsoft", "ice",
        "melbourne", "ngo", "goog", "surf", "kpmg", "save", "capetown",
        "azure", "goo", "fast", "amazon", "download", rootZoneString
    )
)

In [ ]:
plot(
    sortedZones.filter { (z, _) -> domainsWithValidations.contains(IDN.toUnicode(z)) }, listOf(
        "com", "xyz", "site", "amsterdam", "zip", "website", "gov", "paris",
        "education", "link", "support", "ninja", "city", "brussels", "mov",
        "cool", "science", "eco", "pub", "new", "dance", "tel", "domains",
        "press", "adudhabi", "taxi", "ryukyu", "onl", "africa", "vegas",
        "irish", "zuerich", "porn", "adult", "boston", "sucks", "youtube",
        "music", "apple", "honda", "aaa", "aws", "imdb", "now", "deal", "wow",
        "bbc", "dell", "hbo", "safety", "yandex", "playstation", "box", "cern",
        "data", "meme", "stockholm", "tui", "map", "abc", "microsoft", "ice",
        "travel", "melbourne", "ngo", "goog", "surf", "tube", "kpmg", "save",
        "museum", "capetown", rootZoneString
    )
)

In [ ]:
val actuallyUsed =
    sortedZones.map { (z, d) -> Pair(IDN.toUnicode(z), d) }.filter { (z, _) -> domainsWithValidations.contains(z) }
val zoneMinutes = actuallyUsed.map { (z, stat) -> Pair(z, stat.necessarySignatures * (0.424 * (1 - 0.085)) / 60) }
val within30 = zoneMinutes.takeWhile { (_, m) -> m <= 30 }
val within30list = within30.map { (z, _) -> z.split(".") + listOf("") }
val actuallyUsedList = actuallyUsed.map { (z, _) -> z.split(".") + listOf("") }
val amountUsed = udata.filter { u -> actuallyUsedList.contains(u.domainName) }
val amount = udata.filter { u -> within30list.contains(u.domainName) }
println("Zones account for: " + amountUsed.size.toDouble() / udata.size * 100)
println("Within 30 minutes total: " + amount.size.toDouble() / udata.size * 100)
println("Within 30 minutes of used we have zone files for: " + amount.size.toDouble() / amountUsed.size * 100)